In [3]:
# Import necessary libraries for change point detection and Bayesian inference
import pandas as pd
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose


In [4]:
# Load the Brent oil price dataset (make sure you have the correct file path)
df = pd.read_csv("../data/BrentOilPrices.csv")
df['Date'] = pd.to_datetime(df['Date'], format='mixed', dayfirst=True)
df = df.sort_values('Date')
df.reset_index(drop=True, inplace=True)

# Calculate the log returns for stationarity
df['Log_Returns'] = np.log(df['Price'] / df['Price'].shift(1))

# Drop missing values
df = df.dropna()

# Check the first few rows
df.head()


,Date,Price,Log_Returns
1,1987-05-21,18.45,-0.009709
2,1987-05-22,18.55,0.005405
3,1987-05-25,18.60,0.002692
4,1987-05-26,18.63,0.001612
5,1987-05-27,18.60,-0.001612


In [5]:
# Set up the Bayesian Change Point model using PyMC3
with pm.Model() as model:
    # Define the prior for the change point tau
    tau = pm.DiscreteUniform('tau', lower=0, upper=len(df) - 1)
    
    # Define prior for the means before and after the change point
    mu_1 = pm.Normal('mu_1', mu=0, sigma=1)
    mu_2 = pm.Normal('mu_2', mu=0, sigma=1)
    
    # Switch function to select the correct mean based on the time
    mu = pm.math.switch(tau > np.arange(len(df)), mu_1, mu_2)
    
    # Define the likelihood
    sigma = pm.HalfNormal('sigma', sigma=1)
    likelihood = pm.Normal('likelihood', mu=mu, sigma=sigma, observed=df['Log_Returns'])
    
    # Sample from the posterior using MCMC
    trace = pm.sample(2000, return_inferencedata=False)
    
# Check the trace plots
pm.traceplot(trace)
plt.tight_layout()
plt.show()


NameError: name 'pm' is not defined

In [ ]:
# Check the summary of the posterior samples
pm.summary(trace).round(2)

# Extract the change point (tau)
change_point = np.median(trace['tau'])
print(f"Estimated change point: Day {change_point}")


In [ ]:
# Plot the detected change point along with the log returns
plt.figure(figsize=(14, 6))
plt.plot(df['Date'], df['Log_Returns'], label="Log Returns", color='orange')
plt.axvline(df['Date'].iloc[int(change_point)], color='red', linestyle='--', label='Change Point')
plt.title("Change Point Detection in Log Returns")
plt.xlabel("Date")
plt.ylabel("Log Returns")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Load the events dataset to correlate change points with key events
df_events = pd.read_csv('../data/events.csv', parse_dates=['Date'])

# Map events onto price data
df['Event'] = df['Date'].map(df_events.set_index('Date')['Event'])

# Plot the price data with change point and event markers
plt.figure(figsize=(16, 6))
plt.plot(df['Date'], df['Price'], label='Brent Oil Price')
plt.axvline(df['Date'].iloc[int(change_point)], color='red', linestyle='--', label='Change Point')

# Add event annotations
for _, row in df_events.iterrows():
    plt.axvline(x=row['Date'], color='blue', linestyle='--', alpha=0.5)
    plt.text(row['Date'], df['Price'].max()*0.9, row['Event'], rotation=90, fontsize=8, verticalalignment='top')

plt.title('Brent Oil Price with Key Events and Detected Change Point')
plt.xlabel('Date')
plt.ylabel('Price (USD/barrel)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Quantify the impact before and after the change point
before_change = df['Log_Returns'][:int(change_point)].mean()
after_change = df['Log_Returns'][int(change_point):].mean()

# Calculate the change in mean price
price_before = np.exp(before_change)
price_after = np.exp(after_change)

impact = (price_after - price_before) / price_before * 100

print(f"Price before change: {price_before:.2f}")
print(f"Price after change: {price_after:.2f}")
print(f"Impact of change: {impact:.2f}%")


In [ ]:
# Define the Bayesian Change Point Model
with pm.Model() as model:
    tau = pm.DiscreteUniform('tau', lower=0, upper=len(df) - 1)
    mu_1 = pm.Normal('mu_1', mu=0, sigma=1)
    mu_2 = pm.Normal('mu_2', mu=0, sigma=1)
    
    # Switch function to change means based on the change point
    mu = pm.math.switch(tau > np.arange(len(df)), mu_1, mu_2)
    
    # Likelihood function
    sigma = pm.HalfNormal('sigma', sigma=1)
    likelihood = pm.Normal('likelihood', mu=mu, sigma=sigma, observed=df['Log_Returns'])
    
    # Run MCMC to sample from the posterior distribution
    trace = pm.sample(2000, return_inferencedata=False)


In [ ]:
# Check the model's convergence
pm.summary(trace).round(2)

In [ ]:
# Get the change point date and quantify the shift
change_point = np.median(trace['tau'])
print(f"Estimated Change Point: {df['Date'].iloc[int(change_point)]}")

before_change = df['Log_Returns'][:int(change_point)].mean()
after_change = df['Log_Returns'][int(change_point):].mean()

price_before = np.exp(before_change)
price_after = np.exp(after_change)

impact = (price_after - price_before) / price_before * 100
print(f"Price before: {price_before:.2f}")
print(f"Price after: {price_after:.2f}")
print(f"Impact: {impact:.2f}%")
